# YouTube Frame Grabber

Grab frames from a YouTube video at a specific timestamp using `ffmpeg` and `pytubefix`.

- `yt_stream_url(video_url)` — returns a streamable MP4 URL for the given YouTube video
- `grab_frame(url, timestamp)` — grabs a single frame at `timestamp` (seconds) from a stream URL, returns a PIL `Image`
- `grab_frame_tool(url, timestamp, window, n_frames)` — AI tool version: grabs `n_frames` frames spread across a `window`-second window centred on `timestamp`, returns a `ToolResponse` with base64-encoded JPEG images

In [ ]:
from pytubefix import YouTube
def yt_stream_url(video_url):
    return YouTube(video_url).streams.filter(progressive=True, file_extension='mp4').order_by('resolution').last().url

In [ ]:
def yt_chapters(video_url):
    "Return list of chapters for a YouTube video"
    return YouTube(video_url).chapters

In [ ]:
from io import BytesIO
from PIL import Image
import ffmpeg
from lisette.core import *

In [ ]:
def grab_frame(url, timestamp):
    "Grab a frame from a YouTube video at the given timestamp (in seconds)."
    out, _ = ffmpeg.input(url, ss=timestamp).output('pipe:', format='image2pipe', vcodec='mjpeg', vframes=1, **{'q:v': 2}).run(capture_stdout=True, quiet=True)
    return Image.open(BytesIO(out))

In [ ]:
def grab_frames(url, timestamp, window=10, n_frames=5):
    "Grab multiple frames around a timestamp. Returns list of PIL Images."
    start = max(0, timestamp - window/2)
    out, _ = ffmpeg.input(url, ss=start).output('pipe:', format='image2pipe', vcodec='mjpeg', t=window, r=n_frames/window, **{'q:v': 2}).run(capture_stdout=True, quiet=True)
    return [Image.open(BytesIO(jpg)) for jpg in re.findall(b'\xff\xd8.*?\xff\xd9', out, re.DOTALL)]

In [ ]:
def grab_frame_tool(
    url,
    timestamp: float,  # Time in seconds to capture around
    window: float = 4.0,  # Duration in seconds to capture (centered on timestamp)
    n_frames: int = 5 # Number of frames to extract from the window
) -> ToolResponse:
    """
    Grab frames from the current YouTube video around a timestamp.
    Use this tool to find key frames whose visual content adds to the narrative or exposition of the youtube video.
    """
    def img_to_data_url(img):
        buf = BytesIO()
        img.save(buf, format='JPEG')
        return {'type': 'image_url', 'image_url': f"data:image/jpeg;base64,{base64.b64encode(buf.getvalue()).decode()}"}

    try: imgs = [img_to_data_url(img) for img in grab_frames(url, timestamp, window, n_frames)]
    except Exception as e: return f'Frame grab failed: {e}'
    return ToolResponse(imgs)

# Example usage

In [ ]:
# url = yt_stream_url("https://www.youtube.com/watch?v=9oJhdXCLGIw&list=PLFFekKQwSI_2f57wX1gISzZAYgUVWkIiG&index=2")
# url

In [ ]:
# grab_frame(url, 43)